<a href="https://colab.research.google.com/github/agungsugiarto/cnn-batik-classification/blob/master/cnn_batik_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementasi Deep Learning Berbasis Tensorflow Dan Convulutioanal Neural Network (CNN) Untuk Pengenalan Daerah Asal Motif Batik Indonesia

---

by [Agung Sugiarto](https://agungsugiarto.github.io)
/ [GitHub](https://github.com/agungsugiarto/cnn-batik-classification.git)

## Abstrak

Indonesian merupakan bangsa yang terdiri dari berbagai etnik dan memiliki latar belakang budaya yang beraneka ragam. Salah satu hasil kebudayaan masyarakat Indonesia adalah Batik, batik mempunyai banyak macam-macam motif. Corak maupun motif batik tersebut tidak bisa lepas dari unsur-unsur yang melekat dari wilayah asal pembuatannya. Pengetahuan tentang pengenalan motif batik mungkin hanya dimiliki oleh orang-orang tertentu yang memiliki keahlian pada bidang terkait seperti bidang membatik dan tidak semua orang dapat mengenali motif batik tersebut. Namun seiring dengan berkembangnya jaman dan meningkatnya kebutuhan akan informasi mendorong manusia untuk mengembangkan teknologi baru agar pengolahan informasi dapat dilakukan dengan mudah dan cepat. Sehingga dibutuhkan suatu pendekatan dalam penyelesaian permasalahan ini. Salah satu pendekatan dalam pengenalan suatu gambar adalah Deep Learning, merupakan sebuah model neural network yang belakangan ini mulai ramai dikembangkan, hasil dari Deep Learning menunjukkan hasil yang baik dalam meningkatkan akurasi Image Classification atau kasuskasus lainnya. Penelitian ini bertujuan untuk mengetahui cara Deep Learning yaitu Convolutional Neural Network (CNN) dalam melakukan klasifikasi pengenalan daerah asal motif-motif batik.

**Kata Kunci** : *Deep Leraning*, *Convolutional Neural Network*, *Image Classification*, *Batik*.


##Import Dependency##

In [0]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math

This was developed using Python 3.6 (Anaconda) and TensorFlow version:

In [2]:
tf.__version__

'1.13.1'

## Configuration of Neural Network

The configuration of the Convolutional Neural Network is defined here for convenience, so you can easily find and change these numbers and re-run the Notebook.

In [0]:
# Convolutional Layer 1.
filter_size1 = 5          # Convolution filters are 5 x 5 pixels.
num_filters1 = 16         # There are 16 of these filters.

# Convolutional Layer 2.
filter_size2 = 5          # Convolution filters are 5 x 5 pixels.
num_filters2 = 36         # There are 36 of these filters.

# Fully-connected layer.
fc_size = 128             # Number of neurons in fully-connected layer.

## Load Dataset ##